<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/Project01_MCsampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 100  # Number of molecules
L = 10.0  # Box length (Periodic Boundary Conditions applied)
kB = 1.38e-23  # Boltzmann constant (J/K)
T = 300  # Temperature in Kelvin
eps_LJ = 0.994  # Lennard-Jones well depth (kJ/mol)
sigma_LJ = 3.73  # Lennard-Jones sigma parameter (angstrom)
k_theta = 0.5  # Bond angle potential strength

# Initialize positions randomly within the box
pos = np.random.rand(N, 3) * L

# Monte Carlo parameters
num_steps = int(1e5)  # Number of Monte Carlo steps
acceptance = 0
energy_values = []

# Function for Lennard-Jones potential
def lennard_jones_potential(r, eps, sigma):
    """ Compute Lennard-Jones potential without using predefined functions """
    if r == 0:
        return 0  # Avoid division by zero
    r_ratio = sigma / r
    r6 = r_ratio ** 6
    r12 = r6 ** 2
    return 4 * eps * (r12 - r6)

# Function for energy change
def compute_energy_change(pos, i, eps_LJ, sigma_LJ, k_theta, L):
    """ Compute energy change due to trial move without using predefined functions """
    dE = 0
    for j in range(N):
        if i != j:
            rij = pos[i] - pos[j]
            rij = rij - L * np.round(rij / L)  # Apply periodic boundary conditions manually
            r = (rij[0]**2 + rij[1]**2 + rij[2]**2) ** 0.5  # Compute Euclidean distance
            if r > 0:
                dE += lennard_jones_potential(r, eps_LJ, sigma_LJ)
    return dE

# Monte Carlo loop with temperature and density tracking
pressures = []
densities = []
heat_capacities = []
temperatures = np.linspace(200, 500, 5)  # Varying temperatures to check heat capacity
for T in temperatures:
    pos = np.random.rand(N, 3) * L  # Reset positions
    energy_values.clear()  # Clear energy values for each temperature
    acceptance = 0

    # Monte Carlo loop
    for step in range(num_steps):
        # Select a random molecule to move
        i = np.random.randint(N)
        old_pos = np.copy(pos[i])

        # Generate random displacement
        delta = [(np.random.rand() - 0.5) * 0.1 for _ in range(3)]  # Small displacement
        pos[i] = [pos[i][dim] + delta[dim] for dim in range(3)]

        # Apply periodic boundary conditions manually
        pos[i] = [pos[i][dim] % L for dim in range(3)]

        # Compute energy change
        dE = compute_energy_change(pos, i, eps_LJ, sigma_LJ, k_theta, L)

        # Metropolis acceptance criterion
        if dE < 0 or np.random.rand() < np.exp(-dE / (kB * T)):
            acceptance += 1
        else:
            pos[i] = old_pos  # Reject move

        # Store energy for thermodynamic calculations
        energy_values.append(dE)

    # Calculate mean energy and heat capacity
    mean_energy = np.mean(energy_values)
    variance_energy = np.var(energy_values)
    heat_capacity = variance_energy / (kB * T ** 2)

    # Calculate pressure and density
    pressure = N * kB * T / (L ** 3)
    density = N / (L ** 3)

    pressures.append(pressure)
    densities.append(density)
    heat_capacities.append(heat_capacity)

    # Print results for this temperature
    print(f"Temperature: {T} K")
    print(f"Mean Energy: {mean_energy} J")
    print(f"Heat Capacity: {heat_capacity} J/K")
    print(f"Pressure: {pressure} Pa")
    print(f"Density: {density} mol/m^3")
    print(f"Acceptance Ratio: {acceptance / num_steps}")
    print("------------------------------")

# Plotting Equation of State (Pressure vs. Density)
plt.figure(figsize=(8, 6))
plt.plot(densities, pressures, label="Simulated EOS", marker='o')
plt.xlabel("Density (mol/m^3)")
plt.ylabel("Pressure (Pa)")
plt.title("Equation of State (Pressure vs. Density)")
plt.legend()
plt.grid(True)
plt.show()

# Plotting Heat Capacity vs. Temperature
plt.figure(figsize=(8, 6))
plt.plot(temperatures, heat_capacities, label="Simulated Heat Capacity", marker='o')
plt.xlabel("Temperature (K)")
plt.ylabel("Heat Capacity (J/K)")
plt.title("Heat Capacity vs. Temperature")
plt.legend()
plt.grid(True)
plt.show()

UsageError: Cell magic `%%` not found.
